<h1>Comparing Peformance of Single Cache vs Multilayer Cache and Buffers </h1>

---

## **Overview**
This code simulates a **cache-memory hierarchy** to analyze memory access patterns, locality, and performance metrics. It implements multiple cache architectures and models memory accesses using a CPU abstraction.  

---

## **Memory and Cache Modeling**
### **1. Word and Frame Representation**
- **Word:** Represents a `64-bit` data unit.  
- **Frame (Block):** A collection of `16 Words` (1024 bits).  

### **2. Memory Model**
- **Main Memory (`Mainmem`)**: Represents RAM, holding multiple frames.  
- **Cache Memory (`Cachemem`)**: Models cache layers in the hierarchy.  

Each memory layer supports:
- **Read & Write Operations**
- **Cache Miss Handling**
- **Replacement Policies (LRU, Direct Mapped, Associative)**  

### **3. Cache Architectures**
- **Fully Associative Cache:** Any block can be stored anywhere. Uses **LRU replacement**.  
- **Direct Mapped Cache:** A block is mapped to a **fixed** cache location using `(Block Address % Cache Size)`.  
- **Set-Associative Cache (4-Way by Default):** Divides cache into **sets**, where each set holds multiple blocks. Uses **LRU replacement within sets**.  

### **4. CPU Model**
- **Accesses cache memory first**. If a miss occurs, the CPU retrieves data from a lower memory level.  
- **Supports prefetching:** Predicts and loads upcoming memory blocks into cache.  
- **Implements a victim cache and write buffer for optimized memory access.**  

---

## **Memory Access Patterns**
The `GenerateSeq()` function simulates different memory access patterns:
1. **Sequential:** Accesses increasing addresses cyclically.
2. **Random:** Uncorrelated accesses across the address space.
3. **Mixed:** A blend of **sequential locality**, **working-set reuse**, and **random accesses**.

### **Key Enhancements for Locality**
- **Working Set Reuse:** Keeps frequently used addresses for reuse.
- **Stride Variation:** Uses small jumps instead of strict +1 increments.
- **Reduced Random Jumps:** Encourages locality for better cache utilization.

---

## **Performance Analysis**
The `Hierarchy_Analysis()` function evaluates:
1. **Spatial Locality:** Measures the likelihood of accessing consecutive blocks.
2. **Temporal Locality:** Tracks how often addresses are reused over time.

### **Metrics Measured**
- **Hit Rate & Miss Rate**
- **Cache Replacement Analysis (LRU, Write-back)**
- **Read-Write Access Distribution**

---

In [1]:
import string
import inspect  #for docstring formatting
import random
import numpy as np
from copy import deepcopy

In [2]:
#16-Bit Addressing

WORD_SIZE = 64 # in bits
BLOCK_WORDS = 16 # in words

BLOCK_SIZE = BLOCK_WORDS * WORD_SIZE # in bits
CONTENT_SAMPLE_SIZE = 64

DEBUG = False  #set to True for Debug print statements

In [3]:
# Defining Word and Frame/Block

class Word():
    def __init__(self, nullvals = False):
        self.size = WORD_SIZE
        if nullvals: self.data = '0' * (WORD_SIZE // 8)
        else: self.data = ''.join(random.choices(string.ascii_uppercase + string.digits, k=(WORD_SIZE // 8)))

    def __str__(self):
        return self.data

class Frame():
    def __init__(self, nullvals = False):
        self.contents = [Word(nullvals) for i in range(BLOCK_WORDS)]

    def __str__(self):
        outp = str([x.__str__() for x in self.contents])
        return outp

class MemFrame(Frame):
    def __init__(self, address):
        super().__init__()
        self.address = address

class CacheFrame(Frame):
    def __init__(self, nullvals = False, tag = -1, valid = -1, clean = -1, time = -1):
        super().__init__(nullvals)
        self.valid = valid  # set if data in frame is valid
        self.clean = clean  # set if data in cache is same as in memory
        self.tag = tag
        self.time = time # for LRU update policy

In [4]:
#Defining and Configuring Memory

class Memory():
    def __init__(self, memsize, memspeed, name, lower = None):
        self.memsize = self.str_to_bits(memsize)
        self.memspeed = int(memspeed)
        self.name = name
        self.accesses = 0
        self.misses = 0
        self.hitrate = 0
        self.num_blocks = self.memsize // BLOCK_SIZE
        self.lower = lower #when miss occurs, search in lower level memory

    def str_to_bits(self, str):
        parts = str.split()
        xsize = parts[0]
        memsize = 0
        
        if (xsize[-1] == 'b'):
            #bits
            memsize = int(xsize[:-2]) * self.exponent(xsize[-2])
            
        elif (xsize[-1] == 'B'):
            #bytes
            memsize = int(xsize[:-2]) * self.exponent(xsize[-2]) * 8
            
        elif (parts[1] == 'words'):
            #words
            memsize = int(xsize[:-1]) * self.exponent(xsize[-1]) * WORD_SIZE
            
        elif (parts[1] == 'blocks'):
            #blocks
            memsize = int(xsize[:-1]) * self.exponent(xsize[-1]) * BLOCK_SIZE
            
        else :
            print("Input Parse Error")
            return None
        
        return memsize

    def exponent(self, inp):
        if inp == 'k' or inp == 'K': return 2 ** 10
        elif inp == 'm' or inp == 'M': return 2 ** 20
        elif inp == 'g' or inp == 'G': return 2 ** 30
        elif inp == 't' or inp == 'T': return 2 ** 40
        elif inp == 'x' or inp == 'X': return 1
        else: return None

    def reset(self):
        self.accesses = 0
        self.misses = 0
        self.hitrate = 0
        return

def transfer_frame(higher, lower, frame): #frame type transfer from lower to higher
    if isinstance(higher, Cachemem) and isinstance(lower, Cachemem): # Cache to Cache
        newframe = deepcopy(frame)
        return newframe
         
    if isinstance(higher, Mainmem) and isinstance(lower, Mainmem): # RAM to RAM
        newframe = deepcopy(frame)
        return newframe

    if isinstance(higher, Cachemem) and isinstance(lower, Mainmem): # RAM to Cache transfer
        newframe = CacheFrame()
        newframe.tag = frame.address
        newframe.valid = 1
        newframe.clean = 1
        newframe.contents = deepcopy(frame.contents)
        return newframe

    else: # Cache to RAM transfer
        newframe = MemFrame(frame.tag)
        newframe.contents = deepcopy(frame.contents)
        return newframe
        

class Mainmem(Memory):
    def __init__(self, memsize, memspeed, name, lower = None):
        super().__init__(memsize, memspeed, name, lower)
        self.type = 'Random Access Memory'
        self.frames = [MemFrame(i) for i in range(self.num_blocks)]

    def __str__(self):
        str = inspect.cleandoc(f'''
                Name:       {self.name}
                Size:       {self.memsize} bits
                Num Blocks: {self.num_blocks}
                Speed: {self.memspeed}
                Type: {self.type}
                Lower-Heirarchy: {self.lower.name if self.lower else "None"}''')
        return str

    def show_contents(self, sample = False):
        if sample:
            sample_indices = np.random.randint(0, self.num_blocks, CONTENT_SAMPLE_SIZE)
            for indx in sample_indices:
                frame = self.frames[indx]
                print("Block#", frame.address, "Data:", frame) 
            return
        for frame in self.frames:
            print("Block#", frame.address, "Data:", frame) 
        return

    def access(self, address):
        for frame in self.frames:
            if frame.address == address: return frame
        if(DEBUG): print(f"Frame not found in Main Memory object {self.name} -- Accessing Lower Level: {self.lower.name}")
            
        if self.lower == None: return None  #end of the line
            
        frame = self.lower.access(address)
        return frame

    def discard(self, frame):
        address = frame.address
        return

    def update(self, frame):
        address = frame.address
        self.frames[address] = frame
        return


class Cachemem(Memory):
    def __init__(self, memsize, memspeed, name, lower = None):
        super().__init__(memsize, memspeed, name, lower)
        self.frames = [CacheFrame(nullvals = True) for i in range(self.num_blocks)]
        self.discarded_frame = None

    def __str__(self):
        str = inspect.cleandoc(f'''
                Name:       {self.name}
                Size:       {self.memsize} bits
                Num Blocks: {self.num_blocks}
                Speed: {self.memspeed}
                Lower-Heirarchy: {self.lower.name if self.lower else "None"}''')
        return str

    def show_contents(self, sample = False):
        if sample:
            sample_indices = np.random.randint(0, self.num_blocks, CONTENT_SAMPLE_SIZE)
            for indx in sample_indices:
                frame = self.frames[indx]
                print("Block#", indx, "Valid:", frame.valid, "Tag:", frame.tag, "Data:", frame) 
            return
        for indx in range(len(self.frames)):
            frame = self.frames[indx]
            print("Block#", indx, "Valid:", frame.valid, "Tag:", frame.tag, "Data:", frame) 
        return

    def statistics(self):
        print("\n")
        self.hitrate = 1 - (self.misses/self.accesses)
        print("Name: ", self.name)
        print("Type: ", self.type)
        print("Cache Accesses: ", self.accesses) 
        print("Cache Misses: ", self.misses)
        print(f"Hit Rate: {self.hitrate*100:.2f}%")
        print("\n")
        return

    def missrate(self):
        return self.misses/self.accesses

    def reset(self):
        super().reset()
        self.frames = [CacheFrame(nullvals = True) for i in range(self.num_blocks)]
        return


class FullyAssociativeCache(Cachemem):
    def __init__(self, memsize, memspeed, name, lower = None):
        super().__init__(memsize, memspeed, name, lower)
        self.type = "Fully Associative Cache"

    def access(self, tag):
        self.accesses += 1
        for i in range(self.num_blocks):
            frame = self.frames[i]
            if frame.tag == tag and frame.valid == 1:
                self.frames[i].time = self.accesses #updating time for LRU
                if DEBUG: print(f"Cache Hit on {tag}")
                return frame

        self.misses += 1
        if self.lower == None: return None
        
        if DEBUG: print(f"Cache Miss on {tag} -- Accessing Lower Level: {self.lower.name}")
        frame = self.lower.access(tag)
        frame = transfer_frame(self, self.lower, frame)
        frame.valid = 1 # set valid on read from main memory
        frame.clean = 1 # must be clean as it is accessed from main memory
        self.discarded_frame = self.update(frame)
        return frame

    def update(self, newframe):  # LRU Update Policy
        min_time = self.frames[0].time
        for xfr in self.frames:
            min_time = min(min_time, xfr.time)
        for i in range(self.num_blocks):
            if self.frames[i].time == min_time:
                if DEBUG: print(f"Replacing tag: {self.frames[i].tag}")
                newframe.time = self.accesses
                discarded_frame = self.frames[i]

                if discarded_frame.clean == 0 and self.lower: # modified cache frame must be write-back to lower level
                    write_back_frame = transfer_frame(self.lower, self, discarded_frame)
                    self.lower.discard(write_back_frame) #delete old entry if present
                    self.lower.update(write_back_frame)
                    
                self.frames[i] = newframe
                return discarded_frame
        return None
    
    def discard(self, frame):
        tag = frame.tag
        self.accesses += 1
        for i in range(self.num_blocks):
            frame = self.frames[i]
            if frame.tag == tag:
                frame.tag = -1
                frame.valid = -1
                frame.clean = -1
                frame.time = -1
                return True
        return False
            
    def __str__(self):
        return super().__str__() + "\n" + f"Type: {self.type}"

class DirectMappedCache(Cachemem):
    def __init__(self, memsize, memspeed, name, lower = None):
        super().__init__(memsize, memspeed, name, lower)
        self.type = "Direct Mapped Cache"

    def access(self, tag):
        self.accesses += 1
        find_index = tag % self.num_blocks
        candidate_frame = self.frames[find_index]

        if(candidate_frame.tag == tag and candidate_frame.valid == 1):
            #self.frames[find_index].time = self.accesses #updating time for LRU
            if DEBUG: print(f"Cache Hit on {tag}")
            return candidate_frame

        self.misses += 1
        if self.lower == None: return None
                
        if DEBUG: print(f"Cache Miss on {tag} -- Accessing Lower Level: {self.lower.name}")
        frame = self.lower.access(tag)
        frame = transfer_frame(self, self.lower, frame)
        frame.valid = 1 # set valid on read from main memory
        frame.clean = 1 # must be clean as it is accessed from main memory
        self.discarded_frame = self.update(frame)
        return frame

    def update(self, newframe):
        update_indx = newframe.tag % self.num_blocks
        if DEBUG: print(f"Replacing tag: {self.frames[update_indx].tag}")
        newframe.time = self.accesses
        discarded_frame = self.frames[update_indx]

        if discarded_frame.clean == 0 and self.lower: # modified cache frame must be write-back to lower level
            write_back_frame = transfer_frame(self.lower, self, discarded_frame)
            self.lower.discard(write_back_frame) #delete old entry if present
            self.lower.update(write_back_frame)
            
        self.frames[update_indx] = newframe
        return discarded_frame

    def discard(self, frame):
        tag = frame.tag
        self.accesses += 1
        discard_index = tag % self.num_blocks
        frame = self.frames[discard_index]
        if frame.tag == tag:
            frame.tag = -1
            frame.valid = -1
            frame.clean = -1
            frame.time = -1
            return True
        return False
            
    def __str__(self):
        return super().__str__() + "\n" + f"Type: {self.type}"

class KWaySetAssociativeCache(Cachemem):
    def __init__(self, memsize, memspeed, name, lower = None, k = 4):
        super().__init__(memsize, memspeed, name, lower)
        self.type = "4-Way Set Associative Cache"
        self.k = k #setting associativity
        self.num_sets = self.num_blocks // self.k
        
    def access(self, tag):
        self.accesses += 1
        target_set = tag % self.num_sets
        start = target_set * self.k
        end = (target_set * self.k) + self.k
        
        for i in range(start, end):
            frame = self.frames[i]
            if frame.tag == tag and frame.valid == 1:
                self.frames[i].time = self.accesses #updating time for LRU
                if DEBUG: print(f"Cache Hit on {tag}")
                return frame

        self.misses += 1
        if self.lower == None: return None

        if DEBUG: print(f"Cache Miss on {tag} -- Accessing Lower Level: {self.lower.name}")
        frame = self.lower.access(tag)
        frame = transfer_frame(self, self.lower, frame)
        frame.valid = 1 # set valid on read from main memory
        frame.clean = 1 # must be clean as it is accessed from main memory
        self.discarded_frame = self.update(frame)
        return frame

    def update(self, newframe): #LRU update
        target_set = newframe.tag % self.num_sets
        start = target_set * self.k
        end = (target_set * self.k) + self.k
        min_time = self.frames[start].time
        
        for i in range(start, end):
            min_time = min(min_time, self.frames[i].time)
        for i in range(start, end):
            if self.frames[i].time == min_time:
                if DEBUG: print(f"Replacing tag: {self.frames[i].tag}")
                newframe.time = self.accesses
                discarded_frame = self.frames[i]

                if discarded_frame.clean == 0 and self.lower: # modified cache frame must be write-back to lower level
                    write_back_frame = transfer_frame(self.lower, self, discarded_frame)
                    self.lower.discard(write_back_frame) #delete old entry if present
                    self.lower.update(write_back_frame)
                    
                self.frames[i] = newframe
                return discarded_frame
        return None

    def discard(self, frame):
        tag = frame.tag
        self.accesses += 1
        target_set = tag % self.num_sets
        start = target_set * self.k
        end = (target_set * self.k) + self.k
    
        for i in range(start, end):
            frame = self.frames[i]
            if frame.tag == tag:
                frame.tag = -1
                frame.valid = -1
                frame.clean = -1
                frame.time = -1
                return True
                
        return False
            
    def __str__(self):
        return super().__str__() + "\n" + f"Type: {self.type}"

class Buffer(FullyAssociativeCache):
    def __init__(self, memsize, memspeed, name):
        super().__init__(memsize, memspeed, name)
        self.type = "Buffer - Fully Associative"
        self.lower = None


class CPU():
    def __init__(self, name, L1_cache, victim_cache = None, write_buffer = None, inst_prefetch_buffer = None, data_prefetch_buffer = None):
        self.type = "Processor"
        self.name = name
        self.L1_cache = L1_cache
        self.victim_cache = victim_cache
        self.write_buffer = write_buffer
        self.inst_prefetch_buffer = inst_prefetch_buffer
        self.data_prefetch_buffer = data_prefetch_buffer

    def L1_update(self, frame):
        discarded_frame = self.L1_cache.update(frame)
        self.victim_cache.update(discarded_frame)
        return
    
    def access_wrapper(self, address): #this can be updated to be more inline with real world access patterns
        if(DEBUG): print(f"Attemping access of block with address {address}")
            
        frame = None
            
        if self.write_buffer: 
            frame = self.write_buffer.access(address)
            if frame != None:
                if DEBUG: print(f"Found in Write Buffer")
                    
        if frame == None and self.victim_cache: 
            frame = self.victim_cache.access(address)
            if frame != None:
                if DEBUG: print(f"Found in Victim Cache")
                self.L1_update(frame)
                return frame
        
        if frame == None and self.inst_prefetch_buffer: 
            frame = self.inst_prefetch_buffer.access(address)
            if frame != None:
                if DEBUG: print(f"Found in Instruction Prefetch Buffer")
                return frame
                 
        if frame == None and self.data_prefetch_buffer: 
            frame = self.data_prefetch_buffer.access(address)
            if frame != None:
                if DEBUG: print(f"Found in Data Prefetch Buffer")
                return frame

        if(DEBUG): print(f"Checking L1 Cache...")
        frame = self.L1_cache.access(address)
        if frame != None:
            if(DEBUG): print(f"Found in L1 Cache")
            if self.victim_cache != None:    
                discarded_frame = self.L1_cache.discarded_frame
                self.victim_cache.update(discarded_frame)
            return frame

        if(DEBUG): print(f"Memory access attempt failed. Block not available in hierarchy")
        return None

    def access(self, address):
        frame = self.access_wrapper(address)
        newframe = transfer_frame(self.L1_cache, self.L1_cache, frame)
        return newframe

    def write(self, address, updates):  #this can be updated to be more inline with real world access patterns
                                        #updates is a dict: keys are indices in the frame, values are data
        frame = self.access(address)
        if frame == None: 
            if DEBUG: print("Attempting to write to Non-existent frame")
            return
        if self.write_buffer: self.write_buffer.discard(frame) # maybe the frame is already present in write_buffer

        for key, value in updates.items():
            frame.contents[key] = value
        frame.clean = 0 # cache is modified and differs from data in main memory

        # things written to write_buffer are written to main memory when that entry is replaced 
        # from write_buffer causing a cascade-up write back
        replaced_frame = None
        if self.write_buffer:
            replaced_frame = self.write_buffer.update(frame)
            
            if replaced_frame != None:
                discarded_frame = self.L1_cache.update(replaced_frame)
                if self.victim_cache: self.victim_cache.update(discarded_frame)
                
        else:
            discarded_frame = self.L1_cache.update(frame)
            if self.victim_cache: self.victim_cache.update(discarded_frame)
        return

    def prefetch(self, next_address, prev_address):
        if self.inst_prefetch_buffer == None and self.data_prefetch_buffer == None:
            return
        
        if self.inst_prefetch_buffer: 
            frame = self.access(next_address)
            self.inst_prefetch_buffer.update(frame)
            self.L1_cache.discard(frame)
            
        if self.data_prefetch_buffer:
            frame = self.access(prev_address)
            self.data_prefetch_buffer.update(frame)
            self.L1_cache.discard(frame)

        return

In [5]:
def measure_spatial_locality(addr_sequence, cache_block_size):
    block_ids = addr_sequence // cache_block_size
    same_block_count = np.sum(block_ids[:-1] == block_ids[1:]) 
    total_transitions = len(addr_sequence) - 1
    return same_block_count / total_transitions if total_transitions > 0 else 1

def measure_temporal_locality(addr_sequence):
    last_seen = {}
    total_reuse = 0
    reuse_distance = []

    for i, addr in enumerate(addr_sequence):
        if addr in last_seen:
            reuse_distance.append(i - last_seen[addr])
            total_reuse += 1
        last_seen[addr] = i

    if total_reuse == 0:
        return 0  # No reuse found
    return total_reuse / len(addr_sequence)  # Fraction of accesses that were reuses


def GenerateSeqRdOnly(addr_limit, num_accesses = 10000, locality="mixed", working_set_size=100, update_ratio = 0.3):
    if locality == "sequential": # Purely sequential access pattern
        arr = np.arange(num_accesses)  
        arr = arr % addr_limit
        return arr
        
    elif locality == "random":
        arr = np.random.choice(np.arange(num_accesses * 10), num_accesses, replace=False)  # Random addresses
        arr = arr % addr_limit
        return arr
        
    elif locality == "mixed":
        pattern = np.zeros(num_accesses, dtype=int)
        recent_set = np.random.choice(np.arange(num_accesses * 2), working_set_size, replace=False)
        
        for i in range(1, num_accesses):
            # Update working set every 1000 iterations
            if (i%1000 == 0):
                num_to_replace = int(working_set_size * update_ratio)
                # Replace recent_set with random values from pattern
                recent_set[:num_to_replace] = np.random.choice(pattern, num_to_replace, replace=False)
                
            if np.random.rand() < 0.7:  # x% sequential
                pattern[i] = pattern[i - 1] + 1  # Continue sequentially
            elif np.random.rand() < 0.5:  # n% of x% chance to access within a working set (defualt 100 elements)
                pattern[i] = np.random.choice(recent_set)
            else:                         # remaining random
                pattern[i] = np.random.randint(0, num_accesses * 2)  # Random jump
        
        return pattern % addr_limit
        
def Hierarchy_AnalysisRdOnly(CPU, Lowest_Memory, num_accesses = 10000, locality = "mixed"):
    addr_limit = Lowest_Memory.num_blocks * BLOCK_WORDS
    addr_sequence = GenerateSeq(addr_limit, num_accesses, locality)
    print("Spatial Locality Score:", measure_spatial_locality(addr_sequence, BLOCK_WORDS))  # Closer to 1 = better locality
    print("Temporal Locality Score:", measure_temporal_locality(addr_sequence))  

    #simulating accesses
    for addr in addr_sequence:
        tag = addr // BLOCK_WORDS
        if DEBUG: print(f"Attempt Access of addr: {addr} in Block #{tag}\t\t", end = "")
        CPU.access(tag)


def GenerateSeq(addr_limit, num_accesses=10000, locality="mixed", working_set_size=100, update_ratio=0.3):
    if locality == "sequential":  # Purely sequential access pattern
        arr = np.arange(num_accesses) % addr_limit
        return arr

    elif locality == "random":
        arr = np.random.choice(np.arange(num_accesses * 10), num_accesses, replace=False) % addr_limit
        return arr

    elif locality == "mixed":
        pattern = np.zeros(num_accesses, dtype=int)
        recent_set = np.random.choice(np.arange(addr_limit), working_set_size, replace=False)  # More locality

        for i in range(1, num_accesses):
            if i % 1000 == 0:
                num_to_replace = int(working_set_size * update_ratio)
                recent_set[:num_to_replace] = np.random.choice(pattern[:i], num_to_replace, replace=False)

            prob = np.random.rand()
            if prob < 0.93:  # Increase sequential access
                stride = np.random.choice([-1, 1, 2])  # Small strides instead of +1
                pattern[i] = (pattern[i - 1] + stride) % addr_limit  
            elif prob < 0.99:  # More working set usage
                pattern[i] = np.random.choice(recent_set)
            else:  # Reduce randomness
                pattern[i] = np.random.randint(0, addr_limit)

        return pattern


def Hierarchy_Analysis(CPU, Lowest_Memory, num_accesses=10000, locality="mixed", write_probability=0.3):
    addr_limit = Lowest_Memory.num_blocks * BLOCK_WORDS
    addr_sequence = GenerateSeq(addr_limit, num_accesses, locality)

    print("Spatial Locality Score:", measure_spatial_locality(addr_sequence, BLOCK_WORDS))  # Closer to 1 = better locality
    print("Temporal Locality Score:", measure_temporal_locality(addr_sequence))  

    # Simulating accesses (reads and writes)
    for i in range(len(addr_sequence)):
        addr = addr_sequence[i]
        tag = addr // BLOCK_WORDS
        
        if np.random.rand() < write_probability:
            # Generate random content for write operation
            content = {index: ''.join(random.choices(string.ascii_uppercase + string.digits, k=(WORD_SIZE // 8))) for index in np.random.choice(range(16), size=np.random.randint(1, 16), replace=False)}
            if DEBUG: print(f"Writing to Block #{tag} with content {content}")
            CPU.write(tag, content)
        else:
            if DEBUG: print(f"Reading from Block #{tag}")
            CPU.access(tag)

        # prefetching next tag, to be done parallely to currect cache access or write
        next_tag = (tag + 1) % Lowest_Memory.num_blocks
        # prefetching prev tag
        prev_tag = (tag - 1) % Lowest_Memory.num_blocks
        CPU.prefetch(next_tag, prev_tag)

## <<-------------------------------------------------------------------------------------------------------------------------------->>

## <<-------------------------------------------------------------------------------------------------------------------------------->>

<h1>Experiment #1:</h1>

<h2>Experimenting with Single Level - Fully Associative Cache</h2>

In [6]:
RAM0 = Mainmem('64k words', '1', 'RAM0')
print(RAM0)

Name:       RAM0
Size:       4194304 bits
Num Blocks: 4096
Speed: 1
Type: Random Access Memory
Lower-Heirarchy: None


In [7]:
FA_Cache = FullyAssociativeCache('2k words', '1', 'FullyAscCache0', lower = RAM0)
print(FA_Cache)

Name:       FullyAscCache0
Size:       131072 bits
Num Blocks: 128
Speed: 1
Lower-Heirarchy: RAM0
Type: Fully Associative Cache


In [8]:
CPU0 = CPU('CPU0', L1_cache = FA_Cache, victim_cache = None)

In [9]:
DEBUG = False
num_accesses = 100000 

In [10]:
FA_Cache.reset()
Hierarchy_Analysis(CPU0, Lowest_Memory = RAM0, num_accesses = num_accesses, locality = 'mixed')
FA_Cache.statistics()

global_accesses = CPU0.L1_cache.accesses
global_misses = FA_Cache.misses #since L2cache is lowest level of cache here
g_miss_rate = (global_misses / global_accesses) * 100
print(f"Global Miss Rate: {g_miss_rate:.2f}%")

Spatial Locality Score: 0.8572785727857278
Temporal Locality Score: 0.86826


Name:  FullyAscCache0
Type:  Fully Associative Cache
Cache Accesses:  100000
Cache Misses:  9462
Hit Rate: 90.54%


Global Miss Rate: 9.46%


## Global Miss Rate is roughly 10% for a mixed locality scheduling in a single level cache

## <<-------------------------------------------------------------------------------------------------------------------------------->>

<h1>Experiment #2:</h1>

<h2>Testing Improvements with Multi-Level Caching, Victim Cache, Write Buffer</h2>

In [11]:
RAM1 = Mainmem('64K words', '1', 'RAM1')
print(RAM1)

Name:       RAM1
Size:       4194304 bits
Num Blocks: 4096
Speed: 1
Type: Random Access Memory
Lower-Heirarchy: None


In [12]:
L2cache = KWaySetAssociativeCache('16K words', '1', 'L2-4W-Set', lower = RAM1, k = 4)
print(L2cache)

Name:       L2-4W-Set
Size:       1048576 bits
Num Blocks: 1024
Speed: 1
Lower-Heirarchy: RAM1
Type: 4-Way Set Associative Cache


In [13]:
L1cache = DirectMappedCache('2K words', '1', 'L1-Direct', lower = L2cache)
print(L1cache)

Name:       L1-Direct
Size:       131072 bits
Num Blocks: 128
Speed: 1
Lower-Heirarchy: L2-4W-Set
Type: Direct Mapped Cache


In [14]:
victim_cache = Buffer('4x blocks', '1', 'victim-cache-fully-associative')
write_buffer = Buffer('4x blocks', '1', 'write-buffer-fully-associative')
inst_prefetch_buffer = Buffer('4x blocks', '1', 'instruction-prefetch-buffer')
data_prefetch_buffer = Buffer('4x blocks', '1', 'data-prefetch-buffer')

print(victim_cache)

Name:       victim-cache-fully-associative
Size:       4096 bits
Num Blocks: 4
Speed: 1
Lower-Heirarchy: None
Type: Buffer - Fully Associative


In [15]:
CPU1 = CPU('CPU1', L1cache, victim_cache, write_buffer, inst_prefetch_buffer, data_prefetch_buffer)

In [16]:
DEBUG = False
num_accesses = 100000 
caches = [L1cache, L2cache, victim_cache, write_buffer, inst_prefetch_buffer, data_prefetch_buffer]

In [17]:
for cache in caches:
    cache.reset()

Hierarchy_Analysis(CPU1, Lowest_Memory = RAM1, num_accesses = num_accesses, locality = 'mixed')

for cache in caches:
    cache.statistics()

Spatial Locality Score: 0.8544385443854439
Temporal Locality Score: 0.8652


Name:  L1-Direct
Type:  Direct Mapped Cache
Cache Accesses:  348850
Cache Misses:  64494
Hit Rate: 81.51%




Name:  L2-4W-Set
Type:  4-Way Set Associative Cache
Cache Accesses:  71325
Cache Misses:  4241
Hit Rate: 94.05%




Name:  victim-cache-fully-associative
Type:  Buffer - Fully Associative
Cache Accesses:  187351
Cache Misses:  187303
Hit Rate: 0.03%




Name:  write-buffer-fully-associative
Type:  Buffer - Fully Associative
Cache Accesses:  329969
Cache Misses:  187351
Hit Rate: 43.22%




Name:  instruction-prefetch-buffer
Type:  Buffer - Fully Associative
Cache Accesses:  187303
Cache Misses:  89561
Hit Rate: 52.18%




Name:  data-prefetch-buffer
Type:  Buffer - Fully Associative
Cache Accesses:  89561
Cache Misses:  36201
Hit Rate: 59.58%




In [18]:
g_miss_rate = L1cache.missrate() * L2cache.missrate() * 100 #since L2cache is lowest level of cache here
print(f"Global Miss Rate: {g_miss_rate:.2f}%")

Global Miss Rate: 1.10%


## Global Miss Rate is 1% for a mixed locality scheduling for the given multi-level cache hierarchy

<h2>Compared to 10% global miss rate in a single level cache, we are getting only 1% miss rate with this hierarchy, highlighting the effectiveness and efficiency of a multi-level cache and added buffers</h2>